In [1]:
import numpy as np
import pandas as pd
import argparse
import matplotlib.pyplot as plt
# allow multiple outputs in one cell in jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
%qtconsole

In [2]:
norm_data = pd.read_csv('/Users/cgu3/Documents/Crispr/report/all_sample/all_sample.normalized.txt', sep='\t', header=0)
# get number of rows
len(norm_data)

103650

In [3]:
# remove sgRNA and Gene column
norm_data_numeric = norm_data.drop(columns=['sgRNA', 'Gene'])
# find the minimum non-zero value in norm_data
non_zero_min = norm_data_numeric[norm_data_numeric > 0].min().min()
# replace zero value with the minimum non-zero value
norm_data = norm_data.replace(0, non_zero_min)
# calculate log2 ratio for each gene between het and wt
norm_data['LFC'] = np.log10(norm_data['sample_het_1'] / norm_data['sample_wt_1'])
norm_data['LFC'] = np.log10(norm_data['sample_het_6'] / norm_data['sample_wt_6'])

In [5]:
# slice column Gene  sample_het_1 and sample_wt_1
norm_data_1 = norm_data[['sgRNA', 'Gene', 'sample_het_1', 'sample_wt_1', 'LFC']]
norm_data_6 = norm_data[['sgRNA', 'Gene', 'sample_het_6', 'sample_wt_6', 'LFC']]
norm_data_1

,sgRNA,Gene,sample_het_1,sample_wt_1,LFC
0,negative_control_1,negative_control_1,33.209352,0.627188,0.000000
1,negative_control_10,negative_control_10,0.627188,487.132662,-0.780321
2,negative_control_100,negative_control_100,1026.833162,250.041510,0.000000
3,negative_control_1000,negative_control_1000,221.838471,549.892085,2.314081
4,negative_control_1001,negative_control_1001,0.627188,0.627188,0.000000
...,...,...,...,...,...
103645,Zzz3_1,Zzz3,0.627188,374.564174,0.000000
103646,Zzz3_2,Zzz3,1.328374,3108.085696,0.000000
103647,Zzz3_3,Zzz3,2.656748,0.627188,0.000000
103648,Zzz3_4,Zzz3,189.957493,0.627188,3.118402


In [9]:
# manually set args
class Args:
    def __init__(self):
        self.negative_control_keyword = 'negative_control'
        self.output = '/Users/cgu3/Documents/Crispr/report/all_sample/all_sample.normalized.txt'
args = Args()

In [16]:
import random
from random import shuffle
from scipy.stats import mannwhitneyu

# calculate rank using sgRNAs
def rank_test(df, case_group, control_group):
    # df = df[df['treat_mean'] > 20] # only keep sgRNAs with mean read counts > threshold
    df_ntc = df[df['Gene'].str.contains(args.negative_control_keyword)]
    df_targeting = df[~df['Gene'].str.contains(args.negative_control_keyword)]
    # only keep genes with 5 sgRNAs
    df_targeting = df_targeting.groupby('Gene').filter(lambda x: len(x)  == 5)

    ntc_sgRNA_lfc = list(df_ntc['LFC'])
    genes = df_targeting['Gene'].unique()
    num_of_genes = len(genes)
    gene_lfc_p = {}
    for gene in genes:
        df_gene = df_targeting[df_targeting['Gene'] == gene]
        lfc = df_gene.iloc[:5]['LFC'].mean()
        x, pvalue = mannwhitneyu(list(df_gene[case_group]), list(df_gene[control_group]), alternative='two-sided', method='auto')
        gene_lfc_p[gene] = [lfc, pvalue]
    # 
    # random.seed(10)
    # for j in range(num_of_genes):
    #     ntc_sgRNA_p_lfc = list(ntc_sgRNA_lfc)
    #     shuffle(ntc_sgRNA_p_lfc)
    #     ntc_selected = ntc_sgRNA_lfc[:5]
    #     ntc_lfc = np.mean([i[1] for i in sorted(ntc_selected, key=lambda x: x[0])][:5])
    #     x, ntc_pvalue = mannwhitneyu(ntc_selected, ntc_sgRNA_lfc, alternative='two-sided', method='auto')
    #     gene_lfc_p['NTC' + str(j)] = [ntc_lfc, ntc_pvalue]
    return gene_lfc_p
